In [116]:
include("../src/TensorDecomposition.jl")
using LinearAlgebra, LinearSolve

In this notebook we demonstrate the ability to numerically decompose tensors of dimension $n=10, d=4, r=51$ efficiently.  We show this when the $r$ points are drawn generically, so that there is a unique decomposition, and when three points are colinear so there is a one-dimensional space of decompositions.  We remark that because the algorithm is numerical the problem may be ill-conditioned, especially in the nonidentifiable example; robustness of the linear extension procedure merits further consideration.

In [117]:
n = 10
c = 5
r = Int(-0.5*(c+1)*(c-2*(n+1)))

D, Drev = TensorDecomposition.makeDicts(n, 4);
basis_inds = collect(1:r)
basis, basisD = TensorDecomposition.basisFn(basis_inds, Drev);

vars = TensorDecomposition.varTups(basis, n, 4)
eqs1, eqs2 = TensorDecomposition.linEqTups(basisD, n, 4);

r

51

We first draw $\mathbf{z}_1, \dots, \mathbf{z}_{51}\in \mathbb{R}^{11}$ randomly (so they are generic points) and construct the corresponding tensor $\phi\in S^4\mathbb{R}^{11}$.

In [118]:
Z = randn(n+1, r) / sqrt(n)
T = TensorDecomposition.rankedTensor(ones(r), Z, 4; type=eltype(Z));

Tcat = TensorDecomposition.catMat(T, 2)
H0 = Tcat[basis_inds, basis_inds]

Z_ = Z ./ permutedims(Z[1, :]);
permutedims(Z_)

51×11 Matrix{Float64}:
 1.0   -0.503207    1.39263    -0.478684  …   -0.732294    -0.52074
 1.0    1.42981     0.0720141  -0.840908       0.951872     1.03249
 1.0    3.30457    -2.07875     2.83936        1.49843     -0.545794
 1.0   24.7765     20.624       0.468681     -14.7924      -2.90351
 1.0   -0.543813   -0.598014    1.27894        0.838647     0.111397
 1.0   -0.17616    -0.553618    3.1687    …    0.0321998    0.182536
 1.0   -3.0118     -2.79032    -0.430807       1.19805     -7.20829
 1.0    1.58427    -0.187602    1.26429        7.99731     -0.819376
 1.0   -1.15235     1.49358     3.47699        0.715528    -1.1044
 1.0  177.428      73.7681     54.3569       -45.3718     -39.914
 ⋮                                        ⋱                 ⋮
 1.0    4.82139    -4.11108    -2.55242        5.52564      5.65963
 1.0    1.24343    -1.24398    -1.37919        0.621779    -0.507335
 1.0   -3.46342     1.09064     1.46584        0.784559     2.18222
 1.0   -0.0293584   0.21988  

In [119]:
A, b = TensorDecomposition.linearSystem(T, H0, basis_inds, basisD, D, vars, eqs1, []; type=eltype(T));

A_ = Matrix(copy(A))
foreach(TensorDecomposition.normalize!, eachcol(A_));
foreach(TensorDecomposition.normalize!, eachrow(A_));
svdvals(A_)

1526-element Vector{Float64}:
 4.475324834745422
 4.376994962013224
 4.039420182943549
 3.9250169088152336
 3.8964498440904443
 3.7735937629094187
 3.7374508768230084
 3.6480108996843525
 3.5895656408923693
 3.543569122510014
 ⋮
 0.0009666513553210438
 0.0009028350374100346
 0.0006216676697402708
 0.0004342958077539209
 0.00036969997974952996
 0.00025321547613656046
 0.00019261902762483693
 3.6065598321811375e-5
 1.0402936187963716e-5

We can conclude from the above that the matrix $A$ is full column rank.  We solve for the extension.

In [120]:
prob = LinearProblem(A, b)
sol = solve(prob)
solDict = Dict([v => s for (v, s) in zip(vars, sol.u)]);
Ms = TensorDecomposition.multMatrices(T, basis, solDict, D, H0)
lhat, Zhat = TensorDecomposition.obtainDecomp(T, Ms);

In [121]:
maximum(abs.(T-TensorDecomposition.rankedTensor(lhat, Zhat, 4, type=eltype(Zhat))))/maximum(abs.(T))

3.389688170634385e-8

Now we let $\mathbf{z}_{r}=0.2\mathbf{z}_{1}-1.5\mathbf{z}_{2}$, and leave $\mathbf{z}_3, \dots, \mathbf{z}_{r-1}$ generic; we form the corresponding tensor $\phi$.

In [122]:
t1 = 0.2
t2 = -1.5

Z[:, r] = t1*Z[:, 1] + t2*Z[:, 2]
T = TensorDecomposition.rankedTensor(ones(r), Z, 4; type=eltype(Z));

Tcat = TensorDecomposition.catMat(T, 2)
H0 = Tcat[basis_inds, basis_inds]

A, b = TensorDecomposition.linearSystem(T, H0, basis_inds, basisD, D, vars, eqs1, []; type=eltype(T));

A_ = Matrix(copy(A))
foreach(TensorDecomposition.normalize!, eachcol(A_));
foreach(TensorDecomposition.normalize!, eachrow(A_));
svdvals(A_)

1526-element Vector{Float64}:
 4.013380963339816
 3.6221491054614914
 3.4877630686591217
 3.382877383041719
 3.3612239941319273
 3.220339431541813
 3.0863300254604007
 3.0438788653198166
 2.9879206085369794
 2.96970902971055
 ⋮
 0.003024574698653976
 0.002434179594653445
 0.001971734658422671
 0.0011230670323986777
 0.001000117632770667
 0.0006706530207558581
 0.00016721155130723972
 3.7538727288443125e-5
 5.773480656275296e-16

In contrast to the previous example, we see that the matrix $A$ is not full column rank -- it is instead rank $1525$.  We treat the variable $x_5^2x_{10}^3$ as a free parameter.

In [123]:
XCols = TensorDecomposition.getColSubset(A);
XNonCols = setdiff(1:length(vars), XCols)

vars[XNonCols]

1-element Vector{Any}:
 [0, 0, 0, 0, 2, 0, 0, 0, 0, 3]

In [124]:
params = vars[XNonCols]
paramVals = [1]

prob = LinearProblem(A[:, XCols], b - A[:, XNonCols]*paramVals)
solP = solve(prob)
solDict = Dict([(v => x) for (v, x) in zip(vcat(params, setdiff(vars, params)), vcat(paramVals, solP.u))])

Ms = TensorDecomposition.multMatrices(T, basis, solDict, D, H0)
lhat1, Zhat1 = TensorDecomposition.obtainDecomp(T, Ms);

paramVals

1-element Vector{Int64}:
 1

In [125]:
maximum(abs.(T-TensorDecomposition.rankedTensor(lhat1, Zhat1, 4, type=eltype(Zhat1))))/maximum(abs.(T))

1.4402111784937948e-10

In [126]:
params = vars[XNonCols]
paramVals = [0]

prob = LinearProblem(A[:, XCols], b - A[:, XNonCols]*paramVals)
solP = solve(prob)
solDict = Dict([(v => x) for (v, x) in zip(vcat(params, setdiff(vars, params)), vcat(paramVals, solP.u))])

Ms = TensorDecomposition.multMatrices(T, basis, solDict, D, H0)
lhat2, Zhat2 = TensorDecomposition.obtainDecomp(T, Ms);

paramVals

1-element Vector{Int64}:
 0

In [127]:
maximum(abs.(T-TensorDecomposition.rankedTensor(lhat2, Zhat2, 4, type=eltype(Zhat2))))/maximum(abs.(T))

3.799413413483555e-9

We exhibit two distinct decompositions via two choices of parameters: $1$ and $0$.  The decomposition $\mathbf{z}^1_1, \dots, \mathbf{z}^1_r$ corresponds to $1$ and $\mathbf{z}_1^2, \dots, \mathbf{z}_r^2$ corresponds to $0$.  

This example is particularly interesting because there are nontrivial quadratic relations.  However, choosing parameters in the linear system still suffices -- meaning that the quadratic relations become the constant zero in this parameterization.

In [128]:
Z1inds = []
for i=3:r-1
    match = findall(x -> prod(x), eachcol(abs.(Zhat1 .- Z_[:, i]) .< permutedims(1e-4*ones(r))))
    if length(match) == 1
        push!(Z1inds, match[1])
    else 
        println(match)
    end
end

Z2inds = []
for i=3:r-1
    match = findall(x -> prod(x), eachcol(abs.(Zhat2 .- Z_[:, i]) .< permutedims(1e-4*ones(r))))
    if length(match) == 1
        push!(Z2inds, match[1])
    else 
        println(match)
    end
end

The above code shows that the points $\mathbf{z}_3, \dots, \mathbf{z}_{r-1}$ are fixed in any decomposition of $\phi$.  And below we see that $\mathbf{z}_1^1, \mathbf{z}_2^1, \mathbf{z}_r^1$ and $\mathbf{z}_1^2, \mathbf{z}_2^2, \mathbf{z}_r^2$ lie on the line passing through $\mathbf{z}_1, \mathbf{z}_2$.

In [129]:
maximum(abs.(Z_[:, [1, 2]]*pinv(Z_[:, [1, 2]])*Zhat1[:, setdiff(1:r, Z1inds)]-Zhat1[:, setdiff(1:r, Z1inds)]))

1.1177991865451986e-10

In [130]:
maximum(abs.(Z_[:, [1, 2]]*pinv(Z_[:, [1, 2]])*Zhat2[:, setdiff(1:r, Z2inds)]-Zhat2[:, setdiff(1:r, Z2inds)]))

4.753339388763322e-9